In [1]:
import tensorflow as tf
import sys
import numpy as np

In [2]:
## Sequential model

model_seq = tf.keras.Sequential()
model_seq.add(tf.keras.layers.Input(shape=(10,)))
model_seq.add(tf.keras.layers.Dense(64, activation='relu'))
model_seq.add(tf.keras.layers.Dense(32, activation='relu'))
model_seq.add(tf.keras.layers.Dense(1, activation='softmax'))

model_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,817
Trainable params: 2,817
Non-trainable params: 0
_________________________________________________________________


In [3]:
## Functional model

inputs = tf.keras.layers.Input(shape=(10,))
hidden1 = tf.keras.layers.Dense(64, activation='relu')(inputs)
hidden2 = tf.keras.layers.Dense(32, activation='relu')(hidden1)
outputs = tf.keras.layers.Dense(1, activation='softmax')(hidden2)

model_fun = tf.keras.Model(inputs, outputs)

model_fun.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense_3 (Dense)             (None, 64)                704       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,817
Trainable params: 2,817
Non-trainable params: 0
_________________________________________________________________


In [4]:
## Subclassing model

class mymodel(tf.keras.Model):
    def __init__(self, hidden1, hidden2, outputs):
        super(mymodel, self).__init__()
        
        self.hidden1 = tf.keras.layers.Dense(hidden1, activation='relu')
        self.hidden2 = tf.keras.layers.Dense(hidden2, activation='relu')
        self.outputs = tf.keras.layers.Dense(outputs, activation='softmax')
        
    def call(self, inputs):
        x = self.hidden1(inputs)
        x = self.hidden2(x)
        x = self.outputs(x)
        
        return x
    
model_sub = mymodel(64,32,1)

In [5]:
model_sub.build(input_shape=(1,10))

model_sub.summary()

Model: "mymodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             multiple                  704       
                                                                 
 dense_7 (Dense)             multiple                  2080      
                                                                 
 dense_8 (Dense)             multiple                  33        
                                                                 
Total params: 2,817
Trainable params: 2,817
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_sub.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./best_model2.h5',
                                               save_best_only=True,
                                               save_weight_only=True)
earlystop = tf.keras.callbacks.EarlyStopping(patience=5)

In [7]:
# model_sub.fit(X, y, eopchs=100, batch_size=32, callbacks=[checkpoint, earlystop])

In [8]:
##
model = mymodel(32,16,1)

In [9]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

# print(samples)
target =[[1],[0],[1],[1],[0],[1]]

In [10]:
labels = np.array(target)

In [11]:
## tokenizing
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)

display(samples)
# tokenizer.word_index
sequences = np.array(tokenizer.texts_to_sequences(samples))
display(sequences)

['너 오늘 이뻐 보인다',
 '나는 오늘 기분이 더러워',
 '끝내주는데, 좋은 일이 있나봐',
 '나 좋은 일이 생겼어',
 '아 오늘 진짜 짜증나',
 '환상적인데, 정말 좋은거 같아']

array([[ 4,  1,  5,  6],
       [ 7,  1,  8,  9],
       [10,  2,  3, 11],
       [12,  2,  3, 13],
       [14,  1, 15, 16],
       [17, 18, 19, 20]])

In [12]:
batch_size =2
epochs = 100
vocab_size = len(tokenizer.word_index) +1
emb_size = 128
hidden1 = 128
outputs = 1

In [13]:
class mymodel_samples(tf.keras.Model):
    def __init__(self, vocab_size, emb_size, hidden1, outputs):
        super(mymodel_samples, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, emb_size)
        self.hidden1 = tf.keras.layers.Dense(hidden1, activation='relu')
        self.outputs = tf.keras.layers.Dense(outputs, activation='sigmoid')
        
    def call(self, inputs):
        x = self.emb(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.hidden1(x)
        x = self.outputs(x)
        
        return x
    
model_samples = mymodel_samples(vocab_size, emb_size, hidden1, outputs)

In [14]:
model_samples.compile(loss='binary_crossentropy',
                       optimizer='adam',
                       metrics='acc')
model_samples.fit(sequences, labels, epochs=epochs, batch_size=batch_size, )

Epoch 1/100
3/3 [==============================] - 1s 9ms/step - loss: 0.6915 - acc: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6759 - acc: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6656 - acc: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 6ms/step - loss: 0.6508 - acc: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6356 - acc: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6199 - acc: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6021 - acc: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 6ms/step - loss: 0.5804 - acc: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 6ms/step - loss: 0.5550 - acc: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5277 - acc: 1.0000
Epoch 11/100
3/3 [==============================] - 0s 7ms/step - loss: 0.4993 

3/3 [==============================] - 0s 6ms/step - loss: 0.0012 - acc: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 7ms/step - loss: 0.0011 - acc: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 7ms/step - loss: 0.0011 - acc: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 7ms/step - loss: 0.0011 - acc: 1.0000
Epoch 93/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0011 - acc: 1.0000
Epoch 94/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0010 - acc: 1.0000
Epoch 95/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0010 - acc: 1.0000
Epoch 96/100
3/3 [==============================] - 0s 9ms/step - loss: 9.8229e-04 - acc: 1.0000
Epoch 97/100
3/3 [==============================] - 0s 8ms/step - loss: 9.5946e-04 - acc: 1.0000
Epoch 98/100
3/3 [==============================] - 0s 7ms/step - loss: 9.3494e-04 - acc: 1.0000
Epoch 99/100
3/3 [==============================] - 0s 7ms/step - loss:

In [17]:
from sklearn import datasets
import sklearn
import matplotlib.pyplot as plt

In [18]:
iris_data = sklearn.datasets.load_iris()

In [19]:
X = iris_data['data']
y = iris_data['target']

print(X)
print(y)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [20]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)

(120, 4)


In [22]:
ss = StandardScaler()
ss.fit(X_train)

Train_scaled = ss.transform(X_train)
Test_scaled = ss.transform(X_test)

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [24]:
knn = KNeighborsClassifier()

params = {'n_neighbors': range(1,11)}

gs_knn = GridSearchCV(estimator=knn,
                     param_grid = params,
                     scoring = 'accuracy',
                     cv =5)

gs_knn.fit(Train_scaled, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 11)}, scoring='accuracy')

In [25]:
print(gs_knn.best_estimator_)
print(gs_knn.best_score_)

KNeighborsClassifier(n_neighbors=3)
0.9416666666666667


In [26]:
knn_n8 = KNeighborsClassifier(n_neighbors=9)
knn_n8.fit(Test_scaled, y_test)

KNeighborsClassifier(n_neighbors=9)

In [27]:
print(knn_n8.score(Test_scaled, y_test))

0.9


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
text_data = ['나는 배가 고프다', '내일 점심 뭐먹지', '내일 공부 해야겠다', '점심 먹고 공부 해야지']

In [30]:
count_vector = CountVectorizer()
count_vector.fit(text_data)
count_vector.vocabulary_

{'나는': 2,
 '배가': 6,
 '고프다': 0,
 '내일': 3,
 '점심': 7,
 '뭐먹지': 5,
 '공부': 1,
 '해야겠다': 8,
 '먹고': 4,
 '해야지': 9}

In [31]:
display(text_data)
print(count_vector.transform(text_data).toarray())

tdm_count = count_vector.transform(text_data).toarray()

['나는 배가 고프다', '내일 점심 뭐먹지', '내일 공부 해야겠다', '점심 먹고 공부 해야지']

[[1 0 1 0 0 0 1 0 0 0]
 [0 0 0 1 0 1 0 1 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 1 0 0 1 0 0 1 0 1]]


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
text_data = ['나는 배가 고프다', '내일 점심 뭐먹지', '내일 공부 해야겠다', '점심 먹고 공부 해야지']

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(text_data)

TfidfVectorizer()

In [34]:
tfidf_vector = tfidf_vectorizer.transform(text_data)
tfidf_vector.toarray()

array([[0.57735027, 0.        , 0.57735027, 0.        , 0.        ,
        0.        , 0.57735027, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.52640543, 0.        ,
        0.66767854, 0.        , 0.52640543, 0.        , 0.        ],
       [0.        , 0.52640543, 0.        , 0.52640543, 0.        ,
        0.        , 0.        , 0.        , 0.66767854, 0.        ],
       [0.        , 0.43779123, 0.        , 0.        , 0.55528266,
        0.        , 0.        , 0.43779123, 0.        , 0.55528266]])

In [35]:
!pip install nltk

In [36]:
## English tokenizer
import nltk

In [37]:
nltk.download('all-corpora')
nltk.download('punkt')

[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\

[nltk_data]    |   Package propbank is already up-to-date!
[nltk_data]    | Downloading package pros_cons to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package pros_cons is already up-to-date!
[nltk_data]    | Downloading package ptb to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package ptb is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\82103\AppData\Roaming\nltk_data

True

In [38]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [39]:
sentence = "Natural language processing (NLP) is a subfield of computer science \
information engineering, and artificial intelligence concerned \
with the interactions between computers and human (natural) languages, \
in particular how to program computers to process and analyze \
large amounts of natural language data."

print(sentence)

Natural language processing (NLP) is a subfield of computer science information engineering, and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.


In [40]:
word_token = word_tokenize(sentence)

print(word_token)

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', 'information', 'engineering', ',', 'and', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'interactions', 'between', 'computers', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data', '.']


In [41]:
# sentence.split()
sentence_token = sent_tokenize(sentence)
display(sentence_token)



['Natural language processing (NLP) is a subfield of computer science information engineering, and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.']

In [50]:
! java -version

'java'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [46]:
import sys
sys.version

'3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]'

In [51]:
!pip install ./JPype1-1.1.2-cp38-cp38-win_amd64.whl

Processing c:\users\82103\desktop\nlp\jpype1-1.1.2-cp38-cp38-win_amd64.whl


In [52]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [53]:
import konlpy

In [54]:
from konlpy.tag import Okt

In [55]:
okt = Okt()

In [61]:
text = "한글 자연어 처리는 재밌다 이제부터 열심히 해야지ㅎㅎㅎ"

ko_nouns = okt.nouns(text)
ko_morph = okt.morphs(text)
ko_morph_stem = okt.morphs(text, stem=True)
ko_phrase = okt.phrases(text)
ko_pos = okt.pos(text)
# part of speach, pos tagging
print(ko_nouns)
print(ko_morph)
print(ko_morph_stem)
print(ko_phrase)
print(ko_pos)
# print(type(ko_nouns))

['한글', '자연어', '처리', '이제']
['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '해야지', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '하다', 'ㅎㅎㅎ']
['한글', '한글 자연어', '한글 자연어 처리', '이제', '자연어', '처리']
[('한글', 'Noun'), ('자연어', 'Noun'), ('처리', 'Noun'), ('는', 'Josa'), ('재밌다', 'Adjective'), ('이제', 'Noun'), ('부터', 'Josa'), ('열심히', 'Adverb'), ('해야지', 'Verb'), ('ㅎㅎㅎ', 'KoreanParticle')]


In [63]:
from konlpy.corpus import kolaw, kobill

In [66]:
kolaw.open('constitution.txt').read()

# 1809890.txt ~ 1809899.txt
kobill.open('1809892.txt').read()

'교육공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9892\n\n발의연월일 : 2010.  11.  12.  \n\n발  의  자 : 정의화․이명수․김을동 \n\n이사철․여상규․안규백\n\n황영철․박영아․김정훈\n\n김학송 의원(10인)\n\n제안이유 및 주요내용\n\n  현행 교육공무원의 육아휴직은 만 6세 이하의 초등학교 취학 전 자\n\n녀의 육아를 위한 경우로 한정되어 있어 초등학교 취학 후 등하교 및 \n\n방과 후 양육 등에 어려움이 많고, 저학년 자녀 혼자 등하교를 하거나 \n\n어른 없는 집에서 지내다가 성폭력 범죄 등 흉악범죄의 피해자가 되\n\n고 있음. \n\n  이에 육아휴직 가능 시기를 만 8세 이하의 자녀로 확대하여 자녀가 \n\n초등학교에 입학하여 학교생활에 순조롭게 적응할 수 있는 나이까지 \n\n교육공무원이 자녀양육을 위해서 휴직할 수 있도록 하려는 것임(안 제\n\n44조제1항제7호). \n\n- 1 -\n\n\x0c법률  제        호\n\n교육공무원법 일부개정법률안\n\n교육공무원법 일부를 다음과 같이 개정한다.\n\n제44조제1항제7호 중 “자녀(만 6세 이하의 초등학교 취학 전 자녀)”를 \n\n“만 8세 이하(취학 중인 경우에는 초등학교 2학년 이하를 말한다)의 자\n\n녀”로 한다.\n\n제45조제1항제6호 중 “1년 이내로 한다. 다만, 여자 교육공무원의 경\n\n우 任用權者는 2年의 범위내에서 그 休職期間을 연장할 수 있다.”를 \n\n“1년 이내(여자 교육공무원 3년 이내)로 하되, 분할하여 휴직할 수 있\n\n다.”로 한다.\n\n부      칙\n\n이 법은 공포한 날부터 시행한다.\n\n- 3 -\n\n\x0c신 ·구조문대비표\n\n현      행\n\n개   정   안\n\n第44條(休職) ① 敎育公務員이 다\n\n第44條(休職) ① ---------------\n\n음 각 호의 어느 하나에 해당\n\n----------------------------\n\n하

In [106]:
## Re

a = ' Natural language '

# len(a)
# type(a)
# print(a.count('la'))
# print(a.find('a'))
# print(a.find('al'))
# print(a.find('w'))

## join
# print(" ".join('kkkk'))
# print(",".join('kkkk'))

## upper / lower
# print(a.lower())
# print(a.upper())

##
# print(a.strip())
# print(a.rstrip())
# print(a.lstrip())

##
print(a.replace('a','k'))
print(a.split())

 Nkturkl lkngukge 
['Natural', 'language']


In [111]:
print('I eat %d apples' % 3)
print('I eat %s apples' % 5)
print('I eat %d apples on %s' % (5, 'Fridays') )

I eat 3 apples
I eat 5 apples
I eat 5 apples on Fridays


In [112]:
import re

In [119]:
## 메타문자 "." \n 빼고 모든 문자 매칭

print(re.search("ab", "aabadc"))
print(re.search("a.b", "aabadc"))
print(re.search("a.b", "axbawbc"))
index = re.search("a.b", "axbawbc")
print(index.start())
print(index.end())

<re.Match object; span=(1, 3), match='ab'>
<re.Match object; span=(0, 3), match='aab'>
<re.Match object; span=(0, 3), match='axb'>
0
3


In [124]:
## 문자클래스 [.]

print(re.search('a[.]b',"awbawbc"))
print(re.search("a[.]b", "a.bawbc"))

None
<re.Match object; span=(0, 3), match='a.b'>


In [129]:
## 반복메타문자 *, 0 또는 있거나

print(re.search('a*b',"aabaabc"))
print(re.search('a*b',"ababc"))
print(re.search('a*b','aaaaaaaaaaaabaaaaaaaaaaccaaab'))
print(re.search('a*b',"babc"))

<re.Match object; span=(0, 3), match='aab'>
<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 13), match='aaaaaaaaaaaab'>
<re.Match object; span=(0, 1), match='b'>


In [131]:
## 반복메타문자 + 문자가 1 이상

print(re.search('a+b',"aabaadc"))
print(re.search('a+b',"abadc"))
print(re.search('a+b','aaaabaaaadcaab'))
print(re.search('a+b',"badc"))

<re.Match object; span=(0, 3), match='aab'>
<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 13), match='aaaaaaaaaaaab'>
None


In [134]:
## 반복 {m, n}

print(re.search('a{2}b',"aabaadc"))
print(re.search('a{1}b',"abadc"))

print(re.search('a{1,3}b','aaaabaaaadcaab'))
print(re.search('a{1,4}b',"badc"))

<re.Match object; span=(0, 3), match='aab'>
<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(1, 5), match='aaab'>
None


In [137]:
## 반복메타문자 ? 문자 {0,1} 이상

print(re.search('a?b',"aabaadc"))
print(re.search('a?b',"abadc"))
print(re.search('a?b','aaaabaaaadcaab'))
print(re.search('a?b',"badc"))

<re.Match object; span=(1, 3), match='ab'>
<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(3, 5), match='ab'>
<re.Match object; span=(0, 1), match='b'>


In [138]:
p = re.compile('ab*')
print(type(p))

<class 're.Pattern'>


In [150]:
## match: 문자의 첫열
## search: 문자열 전체
p = re.compile('[a-z]+')
print(p.match('python'))
print(p.match('python').group())
print(p.match('python').start())
print(p.match('python').end())
print(p.match('python 3'))

<re.Match object; span=(0, 6), match='python'>
python
0
6
<re.Match object; span=(0, 6), match='python'>


In [153]:
p = re.compile('sk?t')

p.match('string goes here')

<re.Match object; span=(0, 2), match='st'>

In [155]:
p = re.compile('[a-z]+')

result = p.findall('life is too short')
print(result)

['life', 'is', 'too', 'short']


In [157]:
result = p.finditer('life is too short')
print(result)

for i in result:
    print(i)

<re.Match object; span=(0, 4), match='life'>
<re.Match object; span=(5, 7), match='is'>
<re.Match object; span=(8, 11), match='too'>
<re.Match object; span=(12, 17), match='short'>


In [163]:
p = re.compile('[a-z]+', re.I)
print(p.match('python'))
print(p.match('Python'))
print(p.match('PYTHON'))

<re.Match object; span=(0, 6), match='python'>
<re.Match object; span=(0, 6), match='Python'>
<re.Match object; span=(0, 6), match='PYTHON'>


In [176]:
p = re.compile('^python\s\w+')
data = 'python is my life python'
p.findall(data)

['python']

In [181]:
p = re.compile('abc|de')

p.search('bbabcaade')

<re.Match object; span=(2, 5), match='abc'>

In [183]:
p =re.compile('^Life')

print(p.search('Life is too short'))
print(p.search('Too short is my Life'))

<re.Match object; span=(0, 4), match='Life'>
None


In [184]:
p = re.compile('Life$')

print(p.search('Life is too short'))
print(p.search('Too short is my Life'))

None
<re.Match object; span=(16, 20), match='Life'>


In [185]:
p = re.compile('(ABC)+')

p.search('ABCABCABC OK?')

<re.Match object; span=(0, 9), match='ABCABCABC'>

In [220]:
text = 'ParK 010-1234 1234'

## 이름 park
## 전화번호

In [223]:
name = re.compile('[a-z,A-Z]+')
phone_num = re.compile('[0-9,-]+')

print(f'이름 :{name.findall(text)}, 전화번호 :{phone_num.findall(text)}')

이름 :['ParK'], 전화번호 :['010-1234', '1234']


In [219]:
# p = re.compile('\w+')
# n = re.compile('\d{3}[-]\d+[-]\d+')
# print(f'이름 :{p.search(text)}, 전화번호 :{n.search(text)}')

이름 :<re.Match object; span=(0, 4), match='ParK'>, 전화번호 :<re.Match object; span=(5, 18), match='010-1234-1234'>
